# Processo infeccioso pulmonar

Neste estudo de caso-controle realizado no Setor de Anatomia e Patologia do Hospital Heliópolis em São Paulo, no período de 1970 a 1982, vamos analisar dados de pacientes com processo infeccioso pulmonar e ajustar um modelo logístico com efeitos principais.

Os dados consistem em 175 pacientes classificados segundo as seguintes variáveis:
- Y: tipo de tumor (1: maligno, 0: benigno)
- IDADE: idade em anos
- SEXO: sexo (0: masculino, 1: feminino)
- HL: intensidade da célula histiócitos-linfócitos (1: ausente, 2: discreta, 3: moderada, 4: intensa)
- FF: intensidade da célula fibrose-frouxa (1: ausente, 2: discreta, 3: moderada, 4: intensa)

Vamos ajustar um Modelo Logístico para investigar a relação entre as variáveis explicativas e a chance de processo infeccioso pulmonar ser maligno.


In [25]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf


data = pd.read_csv('canc3.dat', sep='\t', names=['tipo', 'idade', 'sexo', 'hl', 'ff'])

data['sexo'] = pd.Categorical(data['sexo'])
data['hl'] = pd.Categorical(data['hl'])
data['ff'] = pd.Categorical(data['ff'])

In [48]:
import pandas as pd
read_file = pd.read_csv("canc3.dat",sep=" ::",header=None)
#read_file.to_csv ('my_file.csv', index=None)

/tmp/ipykernel_1623626/219874301.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  read_file = pd.read_csv("canc3.dat",sep=" ::",header=None)


## Fundamentação teórica

O Modelo Logístico é um tipo específico de Modelo Linear Generalizado (GLM). A variável resposta é binária, e a distribuição usada é a distribuição binomial. A escolha da distribuição binomial é justificada porque ela é adequada para modelar variáveis binárias, como a presença ou ausência de um evento.

O Modelo Logístico utiliza a função de ligação logit:

$$
g(\mu) = logit(\mu) = log\left(\frac{\mu}{1 - \mu}\right)
$$

A parte sistemática do modelo logístico é dada por:

$$
\eta = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + ... + \beta_p X_p
$$

No caso específico do problema apresentado, temos as seguintes variáveis explicativas: IDADE, SEXO, HL e FF. Portanto, o preditor linear η é dado por:

$$
\eta = \beta_1 + \beta_2 \cdot IDADE + \beta_3 \cdot SEXO + \sum_{i=1}^{4} \beta_{4i} \cdot HL_i+ \sum_{i=1}^{4} \beta_{5i} \cdot FF_i
$$


$\eta$ (eta): $\eta$ é a combinação linear das variáveis independentes (ou preditoras) e seus respectivos coeficientes. Em outras palavras, é a soma ponderada das variáveis, levando em consideração a importância (peso) de cada variável no modelo.

Onde $\beta_1$ é o coeficiente da constante (intercept), $\beta_2$, $\beta_3$, $\beta_{4i}$ e $\beta_{5i}$ são os coeficientes das variáveis IDADE, SEXO, HL e FF, respectivamente.

$Pr\{Y = 1 | \eta\}$: Essa é a probabilidade condicional de Y = 1 (evento de interesse) dado $\eta$. Em outras palavras, é a probabilidade de o evento ocorrer (Y = 1) com base nos valores das variáveis preditoras e seus coeficientes.

${1 + e^{-\eta}}^{-1}$: Essa é a função logística ou sigmóide. A função logística transforma os valores de $\eta$ (que podem variar de -∞ a +∞) em uma probabilidade que varia de 0 a 1. A função é dada por:

$$Pr\{Y = 1 | \eta\} = \frac{1}{1 + e^{-\eta}}$$

Aqui, $e^{-\eta}$ é a exponencial de $-\eta$, que é uma função matemática que cresce rapidamente à medida que $\eta$ aumenta.

Agora, juntando todas as partes:

Calcule $\eta$ com base nos valores das variáveis preditoras e seus coeficientes.
Aplique a função logística a $\eta$ para obter a probabilidade condicional de Y = 1.

Assim, a fórmula apresentada é usada para estimar a probabilidade de o evento de interesse (Y = 1) ocorrer com base nos valores das variáveis preditoras e seus coeficientes em uma regressão logística.

Agora vamos ajustar o modelo logístico aos dados e analisar os resultados.


In [4]:
formula = 'tipo ~ sexo + idade + hl + ff'
logistic_model = smf.glm(formula, data=data, family=sm.families.Binomial())
fit = logistic_model.fit()
print(fit.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   tipo   No. Observations:                   35
Model:                            GLM   Df Residuals:                       26
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3.6972
Date:                Fri, 28 Apr 2023   Deviance:                       7.3945
Time:                        00:39:48   Pearson chi2:                     7.95
No. Iterations:                    22   Pseudo R-squ. (CS):             0.6848
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.9711   7.95e+04      0.000      1.0

A tabela apresenta os coeficientes estimados, erros padrão, estatísticas z, p-valores e intervalos de confiança a 95% para cada variável no modelo de regressão logística ajustado. A seguir, interpretamos os resultados de cada variável:

**Sexo**: O coeficiente estimado para o sexo feminino (sexo[T.2]) é 7.1489, com um erro padrão de 3.995. 
O valor z é 1.789 e o p-valor associado é 0.074, indicando que a chance de processo infeccioso maligno
 é maior para o sexo feminino em comparação ao masculino, embora a relação não seja estatisticamente significativa ao nível de 5%.

**Intensidade da célula histiócitos-linfócitos (HL)**: Os coeficientes estimados para as categorias 2, 3 e 4 de HL são -1.2851, -31.2519 e -51.2723, respectivamente. No entanto, os p-valores associados a esses coeficientes são todos muito próximos de 1, indicando que a relação entre a intensidade de HL e o processo infeccioso pulmonar não é estatisticamente significativa.

**Intensidade da célula fibrose-frouxa (FF)**: Os coeficientes estimados para as categorias 2, 3 e 4 de FF são -2.1470, 3.6237 e 4.9491, respectivamente. Assim como para a variável HL, os p-valores associados a esses coeficientes são altos, indicando que a relação entre a intensidade de FF e o processo infeccioso pulmonar não é estatisticamente significativa.

**Idade**: O coeficiente estimado para a idade é 0.2948, com um erro padrão de 0.134. O valor z é 2.196 e o p-valor associado é 0.028, indicando que a chance de processo infeccioso maligno aumenta significativamente com a idade.

Em resumo, a análise dos resultados mostra que a chance de processo infeccioso maligno é maior para o sexo feminino em comparação ao masculino e aumenta significativamente com a idade. No entanto, a relação entre as intensidades das células HL e FF e o processo infeccioso pulmonar não é estatisticamente significativa.


## Resultados

As estimativas dos parâmetros e seus erros padrão aproximados são apresentados na tabela abaixo:

|    Efeito    | Estimativa |   Erro Padrão Aproximado   |
|:------------:|:----------:|:--------------------------:|
|  Constante   |  10.9711   |         7.95e+04           |
|     Sexo     |   7.1489   |           3.995            |
|     Idade    |   0.2948   |           0.134            |
|     HL(2)    |  -1.2851   |         8.18e+04           |
|     HL(3)    | -31.2519   |         7.95e+04           |
|     HL(4)    | -51.2723   |         9.58e+04           |
|     FF(2)    |  -2.1470   |           4.969            |
|     FF(3)    |   3.6237   |           4.627            |
|     FF(4)    |   4.9491   |           5.048            |


O desvio do modelo é de D(y; µ̂) = 157,40 (166 graus de liberdade), indicando um ajuste adequado. Há indícios de que a chance de processo infecioso maligno seja maior para o sexo feminino do que para o sexo masculino. Notamos também que a chance de processo maligno aumenta significativamente com a idade e há indicações de que tanto para a célula FF quanto para HL a chance de processo maligno diminui à medida que aumenta a intensidade da célula.
